# Before

In [1]:
import torch
print(torch.__version__)

@torch.jit.script
def bias_gelu(x):
    return  x * 0.5 * (1.0 + torch.erf(x * 0.70710678))

@torch.jit.script
def bias_gelu_back(g, x):
    ff = 0.5 * (1. + torch.erf(x * 0.70710678)) + 0.3989423 * x * torch.exp(-0.5 * x * x)
    return ff*g

a = torch.randn(1024*1024*256, device='cuda')
torch.cuda.synchronize()
%timeit torch.nn.functional.gelu(a); torch.cuda.synchronize()
torch.cuda.synchronize()
%timeit bias_gelu(a); torch.cuda.synchronize()

a.requires_grad_(True)
b = torch.nn.functional.gelu(a)
torch.cuda.synchronize()
%timeit b.backward(b, retain_graph=True); torch.cuda.synchronize()

1.8.0a0+2fa91fa
4.33 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.2 ms ± 130 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.5 ms ± 93.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# After

In [1]:
import torch
print(torch.__version__)

@torch.jit.script
def bias_gelu(x):
    return  x * 0.5 * (1.0 + torch.erf(x * 0.70710678))

@torch.jit.script
def bias_gelu_back(g, x):
    ff = 0.5 * (1. + torch.erf(x * 0.70710678)) + 0.3989423 * x * torch.exp(-0.5 * x * x)
    return ff*g

a = torch.randn(1024*1024*256, device='cuda')
torch.cuda.synchronize()
%timeit torch.nn.functional.gelu(a); torch.cuda.synchronize()
torch.cuda.synchronize()
%timeit bias_gelu(a); torch.cuda.synchronize()

a.requires_grad_(True)
b = torch.nn.functional.gelu(a)
torch.cuda.synchronize()
%timeit b.backward(b, retain_graph=True); torch.cuda.synchronize()

1.8.0a0+2fa91fa
5.85 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.15 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.6 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
